<a href="https://colab.research.google.com/github/4nur4g/Google-Colab/blob/master/Malstm_quoraquestionpair.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# 1. download data from kaggle, 
# be sure there is a kaggle.json in your drive,
# 

!pip install kaggle

from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth

auth.authenticate_user()

drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])

filename = "/content/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

    100% |████████████████████████████████| 61kB 2.8MB/s 
    100% |████████████████████████████████| 245kB 5.3MB/s 
  Running setup.py bdist_wheel for kaggle ... - \ done
  Stored in directory: /root/.cache/pip/wheels/8b/21/3b/a0076243c6ae12a6215b2da515fe06b539aee7217b406e510e
  Running setup.py bdist_wheel for python-slugify ... - done
  Stored in directory: /root/.cache/pip/wheels/e3/65/da/2045deea3098ed7471eca0e2460cfbd3fdfe8c1d6fa6fcac92
Successfully built kaggle python-slugify
Download 100%.


In [0]:
# download and unzip to /root/input
!kaggle competitions download -c quora-question-pairs

!unzip sample_submission.csv.zip -d /root/input
!unzip test.csv.zip -d /root/input
!unzip train.csv.zip -d /root/input

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 7, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.6/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.6/dist-packages/kaggle/api/kaggle_api_extended.py", line 113, in authenticate
    self.config_file, self.config_dir))
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.
unzip:  cannot find or open sample_submission.csv.zip, sample_submission.csv.zip.zip or sample_submission.csv.zip.ZIP.
unzip:  cannot find or open test.csv.zip, test.csv.zip.zip or test.csv.zip.ZIP.
unzip:  cannot find or open train.csv.zip, train.csv.zip.zip or train.csv.zip.ZIP.


In [0]:
# 2. download google word2vec pretrained embedding 

!wget -P /root/input/ -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2018-11-24 04:54:15--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.80.27
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.80.27|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘/root/input/GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  66.6MB/s    in 34s     

2018-11-24 04:54:50 (45.8 MB/s) - ‘/root/input/GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [0]:
from time import time
import pandas as pd
import numpy as np
!pip install gensim
from gensim.models import KeyedVectors
import re
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

import itertools
import datetime

from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Input, Embedding, LSTM, Merge
import keras.backend as K
from keras.optimizers import Adadelta
from keras.callbacks import ModelCheckpoint

    100% |████████████████████████████████| 23.6MB 1.3MB/s 
    100% |████████████████████████████████| 1.4MB 13.7MB/s 
    100% |████████████████████████████████| 133kB 26.5MB/s 
    100% |████████████████████████████████| 61kB 19.6MB/s 
    100% |████████████████████████████████| 4.9MB 5.9MB/s 
    100% |████████████████████████████████| 552kB 24.9MB/s 
  Running setup.py bdist_wheel for smart-open ... - \ done
  Stored in directory: /root/.cache/pip/wheels/23/00/44/e5b939f7a80c04e32297dbd6d96fa3065af89ecf57e2b5f89f
  Running setup.py bdist_wheel for bz2file ... - done
  Stored in directory: /root/.cache/pip/wheels/81/75/d6/e1317bf09bf1af5a30befc2a007869fa6e1f516b8f7c591cb9
Successfully built smart-open bz2file


Using TensorFlow backend.


ImportError: ignored

In [0]:
# check we have all the necessary files so far
import os
os.chdir("/root/input")
!ls -al

In [0]:
import nltk
nltk.download('stopwords')
TRAIN_CSV = '/root/input/train.csv'
TEST_CSV = '/root/input/test.csv'
EMBEDDING_FILE = '/root/input/GoogleNews-vectors-negative300.bin.gz'
MODEL_SAVING_DIR = '/root/Quora/'
LOG_DIR="/root/logs"

In [0]:
def text_to_word_list(text):
    ''' 
    Pre process and convert texts to a list of words 
    input: str
    output: list of cleaned word
    '''
    
    text = str(text)
    text = text.lower()

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    text = re.sub("quikly","quickly", text)

    text = text.split()

    return text

In [0]:
# 3. create special embedding for this quora question-pair task.

vocabulary = {} # key is the word, value is the index
inverse_vocabulary = ['<unk>'] # value is the word, index is the index
word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)
# check this https://radimrehurek.com/gensim/models/word2vec.html
# binary = true is because the word2vec file we have has a bin suffix, if it's text file, binary = false
# key is the word, value is a 300 dimensional vector for each word

word2vec.word_vec("test")[:10]

In [0]:
# determine the distribution of the length of questions 
# i guess the original max lenght of question is too large, and we can decrease it to accelerate the training speed

#import matplotlib.pyplot as plt

#plt.hist(train_df.question1.map(lambda x: len(str(x).split())).values, bins = 100)

In [0]:

# 2. read the train and test data

train_df = pd.read_csv(TRAIN_CSV)
test_df = pd.read_csv(TEST_CSV)
train_df.head()
# the reason why we use a vocabulary is that we can build a smaller embedding to accerelate the training 

print("load done")

stops = set(stopwords.words('english'))
outside_words=set()
from tqdm import tqdm
questions_cols = ['question1', 'question2']
for dataset in [train_df, test_df]:
  for index,row in (dataset.iterrows()):
    # row is a Series instance
    # check this https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.DataFrame.iterrows.html
    for question in questions_cols:
      
      q2n = [] 
      count_ = 0
      # convert a question to a integer list, the integer 
      # corresponding to the index in the small vocabulary
      for word in text_to_word_list(row[question]):
        
        
        #if word in stops and word not in word2vec.vocab:
        #  continue 
        #if word in stops:
        # i don't remove stop word here, because the length of a question is usually very short. 
        # And those stop words are useful information, such as what, how, etc. 
        #  continue
        if word not in word2vec.vocab:
          outside_words.add(word)
          continue
          
        # limit the length to 50, this decreased the time to train an epoch from more than 1 hour to 20 minutes.
        if count_ >= 50 :
          continue
        count_ +=1
        
        if word not in vocabulary:
          vocabulary[word] = len(inverse_vocabulary)
          q2n.append(len(inverse_vocabulary))
          inverse_vocabulary.append(word)
        else:
          q2n.append(vocabulary[word])
      
      
      dataset.set_value(index,question,q2n)
      # from the api document, set_value has beed slated for deprecation
      #dataset.at[index,question] = q2n
      # check this
      # https://stackoverflow.com/questions/13842088/set-value-for-particular-cell-in-pandas-dataframe-using-index
      
embedding_dim = 300
embeddings = 1 * np.random.randn(len(vocabulary)+1, embedding_dim)
embeddings[0] = 0
# by using the code above, the first vector of embeddings are all 0.
embeddings[0]

      

In [0]:
# determine the distribution of the length of the questions,


In [0]:
print(outside_words)
# how do we do with these misspelling?



{'90fps', 'pr5', 'gkm', 'blackrookbooks', 'whitelabel', 's7edge', 'daith', 'fbr', 'sentiment140', 'osamu', 'gurugaon', 'anthrapology', 'tushar', 'office365', 'suryakanth', 'crypt8', 'ramtha', 'sany', 'carre', 'andamanbluebay', 'grabpanda', 'literative', 'lxde', 'mehldau', 'marysville', 'soraya', 'ketel', 'mconnect', 'incongrous', 'havmor', 'pruthi', 'theano', 'win10', '42', 'hemachromatosis', 'kanon', 'bataan', 'ladak', 'yongnuo', 'ganesan', 'quechua', 'ghotra', 'jaunsari', 'scottland', 'milanno', 'tokenizes', 'steyn', 'formedvalong', 'ulimo', 'delanie', 'a100mfd', '37whrs', 'bdrip', 'stormwind', 'tunnelling', 'acccounts', 'dfa', 't830mp2', 'carpinteria', 'jakub', 'narmada', 'bidswitch', 'prapare', 'arpa', 'isopropene', 'daijyobu', '528', 'hukumat', 'iiph', 'cvpr', 'ileana', 'kitsound', 'msvcr120', 'durov', 'hukamnama', 'net5', 'streimels', 'reddington', 'gorila', 'cougars69', '212', 'habibie', 'ehime', 'm1a1', 'gowen', 'warg', 'lcs', 'nescafe', 'hidayatullah', '7666', 'kanald', 'telol

In [0]:
for word, index in tqdm(vocabulary.items()):
  if word in word2vec.vocab:
    embeddings[index] = word2vec.word_vec(word)
    
del word2vec
# calculate the max length 

max_seq_length = max(train_df.question1.map(lambda x:len(x)).max(),
                    train_df.question2.map(lambda x:len(x)).max(),
                    test_df.question1.map(lambda x:len(x)).max(),
                    test_df.question2.map(lambda x:len(x)).max())
print(max_seq_length)

100%|██████████| 58706/58706 [00:00<00:00, 243481.66it/s]


50


In [0]:


num_samples = len(train_df)
validation_size = (int)(num_samples * 0.2)
training_size = len(train_df) - validation_size 

X = train_df[questions_cols]
X_test = test_df[questions_cols]
Y = train_df['is_duplicate']

# train test split x and y into train and validation
X_train, X_validation, Y_train, Y_validation = train_test_split(X,Y,test_size = validation_size)

# Split to dicts
X_test = {'left':X_test.question1,'right':X_test.question2}
X_train = {'left':X_train.question1,'right':X_train.question2}
X_validation = {'left':X_validation.question1,'right':X_validation.question2}
#X_test = {'left':X_test.question1,'right':X_test.question2}

# fetch Ys
Y_train = Y_train.values
Y_test = Y_validation.values

# zero padding

for dataset, side in itertools.product([X_test, X_train,X_validation],['left','right']):
  dataset[side] = pad_sequences(dataset[side], maxlen = max_seq_length)

# Make sure everything is ok
assert X_train['left'].shape == X_train['right'].shape
assert len(X_train['left']) == len(Y_train)


In [0]:
train_df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,"[1, 2, 3, 4, 5, 4, 6, 7, 8, 9, 10, 8, 11]","[1, 2, 3, 4, 5, 4, 6, 7, 8, 9, 10]",0
1,1,3,4,"[1, 2, 3, 12, 13, 14, 15, 16]","[1, 17, 18, 19, 3, 20, 21, 22, 3, 13, 14, 15, ...",0
2,2,5,6,"[24, 25, 14, 26, 3, 27, 28, 29, 30, 31, 32, 33]","[24, 25, 29, 27, 34, 35, 5, 36, 37, 38]",0
3,3,7,8,"[39, 40, 14, 41, 42, 43, 24, 25, 14, 44, 45]","[46, 3, 47, 48, 49, 50, 49, 2, 51, 5]",0
4,4,9,10,"[52, 53, 54, 8, 55, 56, 57, 58, 59, 60, 61, 62]","[52, 63, 17, 64, 8, 58, 55]",0


In [0]:
import keras
import tensorflow as tf

n_hidden = 50
gradient_clipping_norm  = 1.25
# what is gradient clipping norm?
# https://www.quora.com/What-is-gradient-clipping-and-why-is-it-necessary

batch_size = 64
n_epoch = 3

def exponent_neg_manhattan_distance(left, right):
  return K.exp(-K.sum(K.abs(left-right),axis = 1, keepdims = True))

# a. input layer
left_input = Input(shape=(max_seq_length,), dtype="int32")
right_input = Input(shape=(max_seq_length,), dtype="int32")

# b. embedding layer
embedding_layer = Embedding(
    input_dim = len(embeddings), # size of the vocabulary, len( a matrix) will 
    # only return its first dimension length. The shape of the embedding matrix is
    # input_dim, output_dim
    
    output_dim = embedding_dim, # 
    weights= [embeddings], # there is not weights in Embedding api
    # https://keras.io/layers/embeddings/
    # but there are examples using pretrained word2vec in the blog below:
    # https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html
    input_length=max_seq_length,
    trainable=False)
# https://keras.io/layers/embeddings/

encoded_left = embedding_layer(left_input)
encoded_right = embedding_layer(right_input)

#
shared_lstm = LSTM(n_hidden)
left_output = shared_lstm(encoded_left)
right_output = shared_lstm(encoded_right)

malstm_distance = Merge(mode = lambda x:exponent_neg_manhattan_distance(x[0],x[1]), 
                        output_shape=lambda x: (x[0][0], 1))([left_output, right_output])
# several exampls about Merge
# https://statcompute.wordpress.com/2017/01/08/an-example-of-merge-layer-in-keras/

malstm = Model([left_input, right_input], [malstm_distance])
malstm.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 50)           0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 50)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 50, 300)      17612100    input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 50)           70200       embedding_1[0][0]                
          

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:43: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


In [0]:
optimizer = Adadelta(clipnorm = gradient_clipping_norm)
# What is Adadelta optimizer?
# https://keras.io/optimizers/#adadelta
# the advantage of this is we dont need to set a learning-rate for the optimizer. 
# Adadelta is a more robust extension of Adagrad that adapts learning rates based 
# on a moving window of gradient updates, instead of accumulating all past gradients.

# https://arxiv.org/pdf/1211.5063.pdf

# https://www.quora.com/What-is-gradient-clipping-and-why-is-it-necessary
# here, we just need to know, gradient clipping can make the converge faster, it's helpful to limit the gradient to control the exploding of gradient

In [0]:
malstm.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['accuracy'])

from keras.callbacks import TensorBoard
tensorboard = TensorBoard(log_dir=LOG_DIR.format(time()))


malstm_trained = malstm.fit([X_train['left'], X_train['right']], Y_train, batch_size=batch_size, nb_epoch=n_epoch,
                            validation_data=([X_validation['left'], X_validation['right']], Y_validation), callbacks=[tensorboard])


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Train on 323432 samples, validate on 80858 samples
Epoch 1/3
 42880/323432 [==>...........................] - ETA: 19:08 - loss: 0.2066 - acc: 0.6880

323392/323432 [============================>.] - ETA: 0s - loss: 0.1802 - acc: 0.7319

323432/323432 [==============================] - 1417s 4ms/step - loss: 0.1802 - acc: 0.7319 - val_loss: 0.1686 - val_acc: 0.7506
Epoch 2/3
 20288/323432 [>.............................] - ETA: 20:53 - loss: 0.1655 - acc: 0.7598

323392/323432 [============================>.] - ETA: 0s - loss: 0.1623 - acc: 0.7687

323432/323432 [==============================] - 1405s 4ms/step - loss: 0.1623 - acc: 0.7687 - val_loss: 0.1611 - val_acc: 0.7755
Epoch 3/3
 20288/323432 [>.............................] - ETA: 20:35 - loss: 0.1557 - acc: 0.7821

323392/323432 [============================>.] - ETA: 0s - loss: 0.1554 - acc: 0.7821

323432/323432 [==============================] - 1418s 4ms/step - loss: 0.1554 - acc: 0.7821 - val_loss: 0.1559 - val_acc: 0.7845


In [0]:
# Extend: .how to predict using the trained keras model?

y_test=malstm.predict([X_test['left'][0:10],X_test['right'][0:10]])
y_test>0.5


array([[False],
       [False],
       [ True],
       [False],
       [ True],
       [False],
       [False],
       [ True],
       [False],
       [False]])

In [0]:
# tensorboard support, 
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)
get_ipython().system_raw('./ngrok http 6006 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2018-08-01 20:25:11--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.207.39.76, 52.203.53.176, 52.203.66.95, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.207.39.76|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5363700 (5.1M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]   5.11M  21.9MB/s    in 0.2s    

2018-08-01 20:25:11 (21.9 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [5363700/5363700]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
http://9a932695.ngrok.io


In [0]:
sub = pd.DataFrame()
p_test = malstm.predict([X_test['left'][0:10],X_test['right'][0:10]])

p_test

array([[0.07054728],
       [0.3889698 ],
       [0.46472833],
       [0.07809712],
       [0.6397767 ],
       [0.10842681],
       [0.48167175],
       [0.7641893 ],
       [0.45490623],
       [0.09487382]], dtype=float32)

In [0]:
test_df2 = pd.read_csv(TEST_CSV)


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
test_df2['question1'][7]

'By scrapping the 500 and 1000 rupee notes, how is RBI planning to fight against issue black money?'

In [0]:
test_df2['question2'][7]

'How will the recent move to declare 500 and 1000 denomination lewin illegal will curb black money?'

In [0]:
from google.colab import files

sub.is_duplicate = sub.is_duplicate*1
sub.to_csv('simple_xgb.csv', index=False)

In [0]:
!ls -al

total 2261408
drwxr-xr-x 2 root root       4096 Jul 27 05:14 .
drwx------ 1 root root       4096 Jul 26 21:47 ..
-rw-r--r-- 1 root root 1647046227 Mar  5  2015 GoogleNews-vectors-negative300.bin.gz
-rw-r--r-- 1 root root   70743472 Jul 27 05:14 model.h5
-rw-r--r-- 1 root root   22346871 Jun 11 11:23 sample_submission.csv
-rw-r--r-- 1 root root   34523673 Jul 27 04:54 simple_xgb.csv
-rw-r--r-- 1 root root  477592983 Jun 11 11:23 test.csv
-rw-r--r-- 1 root root   63399110 Jun 11 11:23 train.csv


In [0]:
malstm.save_weights("model.h5")

In [0]:
files.download('simple_xgb.csv')
files.download('model.h5')